In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from exo_finder.default_datasets import exo_dataset

# Statistics of stars hosting exoplanets

In [ ]:
exo_db = exo_dataset.load_known_exoplanets_dataset(with_gaia_star_data=True).get_default_records()
gaia_db = exo_dataset.load_gaia_dataset_of_known_exoplanets()
exo_stars = pd.merge(exo_db.to_pandas(), gaia_db.to_pandas(), on="gaia_id", how="inner")
exo_stars

#### Scatter of stars in relation of their habitable zone

In [ ]:
def plot_habitable_zone(ax):
    """
    Visualizes exoplanets and their position relative to the habitable zone.
    ref: https://www.planetarybiology.com/calculating_habitable_zone.html
    """
    
    # Remove rows with NaNs in relevant columns
    valid_data = exo_stars.dropna(subset=["pl_orbsmax", "inner_hz", "outer_hz"])
    magnitudes = valid_data["lum_flame"]
    
    # Plot the habitable zone regions
    mags = np.linspace(magnitudes.min(), magnitudes.max(), 100)
    inners = np.sqrt(mags/1.1)
    outers = np.sqrt(mags/0.53)
    
    ax.fill_between(mags, outers, 1e4, color='blue', alpha=0.2, label="Too Cold")
    ax.fill_between(mags, inners, outers, color='green', alpha=0.4, label="Habitable Zone")
    ax.fill_between(mags, 10**-2.5, inners, color='red', alpha=0.2, label="Too Hot")
    
    ax.scatter(magnitudes, valid_data["pl_orbsmax"], color='blue', label="Exoplanets", s=2)
    ax.set(
        xlabel=r"Star's luminosity [$L_\odot$]",
        ylabel="Distance from star [au]",
        yscale="log",
        xscale="log",
        # title="Exoplanets and the Habitable Zone",
    )
    ax.legend(loc='upper right')
    
plt.figure(figsize=(9, 6))
plot_habitable_zone(ax=plt.gca())
plt.tight_layout()
plt.show()


## Distribution of stellar parameters

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

def star_radius_distribution(ax):
    data = exo_stars["radius"].dropna().values
    data = data[data < np.quantile(data, 0.97)]
    ax.hist(data, bins=50)
    ax.set(title="Radius", xlabel=r"Radius [$R_\odot$]", ylabel="Count")

def star_mass_distribution(ax):
    data = exo_stars["mass_flame"].dropna().values
    data = data[data < np.quantile(data, 0.99)]
    ax.hist(data, bins=50)
    ax.set(title="Mass", xlabel=r"Mass [$M_\odot$]", ylabel="Count")

def star_temperature_distribution(ax):
    data = exo_stars["teff_mean"].dropna()
    ax.hist(data, bins=50)
    ax.set(title="Effective Temperature", xlabel=r"Temperature [K]", ylabel="Count")


star_radius_distribution(axes[0])
star_mass_distribution(axes[1])
star_temperature_distribution(axes[2])

# plt.suptitle("Distribution of stellar parameters")
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

def star_radius_planet_radius(ax):
    star_radii = exo_stars[["st_rad", "st_rad_gaia"]].mean(axis=1)
    pl_radii = exo_stars["pl_rade"]
    
    sns.scatterplot(x=star_radii, y=pl_radii, ax=ax, alpha=0.5, s=8)
    ax.set(title="Star vs planet radius", xlim=(0, 4), ylim=(0, 25), xlabel=r"Star radius [$R_\odot$]", ylabel=r"Planet radius [$R_\oplus$]")

def star_temp_mass(ax):
    masses = exo_stars[["mass_flame", "st_mass"]].mean(axis=1)
    sns.scatterplot(exo_stars, x="teff_mean", y=masses, hue="spectraltype_esphs", ax=ax, alpha=0.5, s=7)
    ax.set(title="Star temperature vs mass", xlabel=r"Star temperature [K]", ylim=(0, 3), ylabel=r"Star Mass [$M_\odot$]")
    ax.legend(title="Type", markerscale=2)

def star_temp_lum(ax):
    data = exo_stars[["teff_mean", "lum_flame", "spectraltype_esphs"]].dropna()
    sns.scatterplot(data, x="teff_mean", y="lum_flame", hue="spectraltype_esphs", ax=ax, alpha=0.5, s=7)
    ax.set(title="Star temperature vs luminosity", xlabel=r"Star temperature [K]", ylabel=r"Star Luminosity [$\log_{10}L_\odot$]", yscale="log")
    ax.legend(title="Type", markerscale=2)

def star_distance_luminosity(ax):
    sns.scatterplot(exo_stars, x="distance_msc", y="radius", ax=ax, alpha=0.5, s=7)
    ax.set(title="Star distance vs radius", yscale="log", xlabel="Distance from Earth [pc]", ylabel=r"Radius [$R_\odot$]")

star_radius_planet_radius(axes[0])
star_temp_lum(axes[1])
star_distance_luminosity(axes[2])

# plt.suptitle("Relation of stellar and planetary parameters")
plt.tight_layout()
plt.show()